In [1]:
import pandas as pd
import numpy as np
import pyreadr
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import linregress
import os

In [33]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [19]:
data_predict = pd.read_pickle('pickles/data_predict_2009.pkl')
display(data_predict)

,song_id,tag,release_date,release_month,release_year,listen_times,chroma_stft_norm,rmse_norm,spectral_centroid_norm,spectral_bandwidth_norm,...,knn_pred_bin,knn_pred_bin_acc,rf_pred,rf_pred_acc,rf_pred_bin,rf_pred_bin_acc,xgb_pred,xgb_pred_acc,xgb_pred_bin,xgb_pred_bin_acc
1,xNgVaKafcd0,pop,2017-09-08,2017.666667,2017,952.0,0.313235,0.226911,0.325286,0.570343,...,nonhiphop,True,folk,False,nonhiphop,True,3.0,False,nonhiphop,True
2,mSOdaq656ff,rock,2015-11-11,2015.833333,2015,601.0,0.553648,0.305978,0.255246,0.479821,...,nonhiphop,True,rock,True,nonhiphop,True,1.0,False,nonhiphop,True
3,bqv5oMs3aac9,pop,2016-05-24,2016.333333,2016,425.0,0.317953,0.153784,0.248175,0.558457,...,nonhiphop,True,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
4,xLBr1fbf8ac,pop,2015-06-27,2015.416667,2015,42450.0,0.295309,0.174921,0.247695,0.550785,...,nonhiphop,True,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
5,bqwPhDt3af1d,pop,2016-04-14,2016.250000,2016,352.0,0.359844,0.161044,0.276050,0.628411,...,nonhiphop,True,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94811,mSFWvm87f0a,pop,2012-07-12,2012.500000,2012,24.0,0.420810,0.162462,0.345590,0.702055,...,nonhiphop,True,pop,True,nonhiphop,True,NaN,False,nonhiphop,True
94812,xNPVxjbf14e,pop,2015-04-08,2015.250000,2015,172.0,0.302126,0.199702,0.278044,0.568268,...,nonhiphop,True,pop,True,nonhiphop,True,NaN,False,nonhiphop,True
94813,8IAUlNe26fb,pop,2015-04-08,2015.250000,2015,84.0,0.584518,0.210099,0.487939,0.758034,...,nonhiphop,True,hiphop,False,hiphop,False,NaN,False,nonhiphop,True
94814,bCniNa4469b4,pop,2016-04-08,2016.250000,2016,222.0,0.367468,0.317775,0.482698,0.781361,...,nonhiphop,True,pop,True,nonhiphop,True,NaN,False,nonhiphop,True


In [42]:
subg = pd.concat([pd.Series(data_predict.index), data_predict], axis = 1).rename(columns={0:'id'})
display(subg)

,id,song_id,tag,release_date,release_month,release_year,listen_times,chroma_stft_norm,rmse_norm,spectral_centroid_norm,spectral_bandwidth_norm,rolloff_norm,zero_crossing_rate_norm,mfcc1_norm,mfcc2_norm,mfcc3_norm,mfcc4_norm,mfcc5_norm,mfcc6_norm,mfcc7_norm,mfcc8_norm,mfcc9_norm,mfcc10_norm,mfcc11_norm,mfcc12_norm,...,mfcc16_norm,mfcc17_norm,mfcc18_norm,mfcc19_norm,mfcc20_norm,tag_bin,treatment_date,treatment_year,gnb_pred,gnb_pred_acc,gnb_pred_bin,gnb_pred_bin_acc,knn_pred,knn_pred_acc,knn_pred_bin,knn_pred_bin_acc,rf_pred,rf_pred_acc,rf_pred_bin,rf_pred_bin_acc,xgb_pred,xgb_pred_acc,xgb_pred_bin,xgb_pred_bin_acc,id
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,xNgVaKafcd0,pop,2017-09-08,2017.666667,2017.0,952.0,0.313235,0.226911,0.325286,0.570343,0.463045,0.159881,0.802928,0.438278,0.463192,0.548091,0.388213,0.378023,0.495891,0.288580,0.488314,0.653325,0.411552,0.365625,...,0.443594,0.433783,0.280255,0.320974,0.607268,nonhiphop,1.0,1.0,folk,False,nonhiphop,True,pop,True,nonhiphop,True,folk,False,nonhiphop,True,3.0,False,nonhiphop,True,0 0 pop
2,3.0,mSOdaq656ff,rock,2015-11-11,2015.833333,2015.0,601.0,0.553648,0.305978,0.255246,0.479821,0.377546,0.099607,0.824299,0.524473,0.405405,0.709515,0.526825,0.493619,0.486819,0.663650,0.652753,0.769993,0.717449,0.484404,...,0.555597,0.530681,0.281744,0.344796,0.510593,nonhiphop,0.0,0.0,rock,True,nonhiphop,True,rock,True,nonhiphop,True,rock,True,nonhiphop,True,1.0,False,nonhiphop,True,0 0 rock
3,4.0,bqv5oMs3aac9,pop,2016-05-24,2016.333333,2016.0,425.0,0.317953,0.153784,0.248175,0.558457,0.349581,0.093165,0.757307,0.546043,0.536335,0.520767,0.543061,0.461001,0.461963,0.549711,0.498131,0.638340,0.569084,0.339578,...,0.531985,0.452319,0.275470,0.249064,0.486048,nonhiphop,0.0,0.0,folk,False,nonhiphop,True,pop,True,nonhiphop,True,pop,True,nonhiphop,True,1.0,False,nonhiphop,True,0 0 pop
4,5.0,xLBr1fbf8ac,pop,2015-06-27,2015.416667,2015.0,42450.0,0.295309,0.174921,0.247695,0.550785,0.362049,0.090020,0.749085,0.521823,0.549510,0.566503,0.522691,0.396086,0.530403,0.533256,0.584893,0.616224,0.567201,0.407748,...,0.539164,0.399455,0.224764,0.347304,0.591887,nonhiphop,0.0,0.0,folk,False,nonhiphop,True,pop,True,nonhiphop,True,pop,True,nonhiphop,True,1.0,False,nonhiphop,True,0 0 pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94811,NaN,mSFWvm87f0a,pop,2012-07-12,2012.500000,2012.0,24.0,0.420810,0.162462,0.345590,0.702055,0.585147,0.114261,0.790627,0.417874,0.569560,0.537360,0.492187,0.441719,0.504366,0.587671,0.632251,0.654278,0.620151,0.396508,...,0.574771,0.495532,0.282261,0.318730,0.520840,nonhiphop,0.0,0.0,pop,True,nonhiphop,True,pop,True,nonhiphop,True,pop,True,nonhiphop,True,NaN,False,nonhiphop,True,0 0 pop
94812,NaN,xNPVxjbf14e,pop,2015-04-08,2015.250000,2015.0,172.0,0.302126,0.199702,0.278044,0.568268,0.416383,0.107042,0.741939,0.498527,0.551582,0.468532,0.453421,0.442239,0.525247,0.417638,0.698584,0.642637,0.655453,0.445301,...,0.650760,0.502584,0.267097,0.306384,0.557927,nonhiphop,0.0,0.0,folk,False,nonhiphop,True,pop,True,nonhiphop,True,pop,True,nonhiphop,True,NaN,False,nonhiphop,True,0 0 pop
94813,NaN,8IAUlNe26fb,pop,2015-04-08,2015.250000,2015.0,84.0,0.584518,0.210099,0.487939,0.758034,0.738617,0.197534,0.846625,0.301963,0.533344,0.494454,0.448711,0.459509,0.511496,0.571318,0.585940,0.728663,0.668336,0.479782,...,0.634211,0.576412,0.329460,0.324301,0.581597,nonhiphop,0.0,0.0,hiphop,False,hiphop,False,pop,True,nonhiphop,True,hiphop,False,hiphop,False,NaN,False,nonhiphop,True,0 0 pop
94814,NaN,bCniNa4469b4,pop,2016-04-08,2016.250000,2016.0,222.0,0.367468,0.317775,0.482698,0.781361,0.757807,0.186149,0.869541,0.299227,0.569001,0.541548,0.481844,0.440022,0.529405,0.5636

In [31]:
subgenres= data_predict.groupby(['song_id','tag']).size().reset_index().pivot(index = 'song_id', columns = 'tag', values = 0)

In [37]:
display(subgenres)
print(data_predict.index)

tag,folk,hiphop,pop,rock
song_id,,,,
4j46d5e4,NaN,NaN,1.0,NaN
4jBabaec,NaN,NaN,1.0,NaN
4jC543be,NaN,NaN,1.0,NaN
4jE5cc3e,NaN,NaN,1.0,NaN
4jL7d403,NaN,NaN,1.0,NaN
...,...,...,...,...
xOoghub575e,NaN,NaN,1.0,NaN
xOoghvd2ec5,NaN,NaN,1.0,NaN
xOoghwbc25a,NaN,NaN,1.0,NaN


Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               11,
            ...
            94806, 94807, 94808, 94809, 94810, 94811, 94812, 94813, 94814,
            94815],
           dtype='int64', length=69427)


In [36]:
data_predict_subgenres = pd.concat([data_predict, subgenres], axis = 1)
display(data_predict_subgenres)

/home/priscilla/.local/lib/python3.10/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/home/priscilla/.local/lib/python3.10/site-packages/IPython/core/formatters.py:343: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,song_id,tag,release_date,release_month,release_year,listen_times,chroma_stft_norm,rmse_norm,spectral_centroid_norm,spectral_bandwidth_norm,rolloff_norm,zero_crossing_rate_norm,mfcc1_norm,mfcc2_norm,mfcc3_norm,mfcc4_norm,mfcc5_norm,mfcc6_norm,mfcc7_norm,mfcc8_norm,mfcc9_norm,mfcc10_norm,mfcc11_norm,mfcc12_norm,mfcc13_norm,...,mfcc20_norm,tag_bin,treatment_date,treatment_year,gnb_pred,gnb_pred_acc,gnb_pred_bin,gnb_pred_bin_acc,knn_pred,knn_pred_acc,knn_pred_bin,knn_pred_bin_acc,rf_pred,rf_pred_acc,rf_pred_bin,rf_pred_bin_acc,xgb_pred,xgb_pred_acc,xgb_pred_bin,xgb_pred_bin_acc,id,folk,hiphop,pop,rock
1,xNgVaKafcd0,pop,2017-09-08,2017.666667,2017.0,952.0,0.313235,0.226911,0.325286,0.570343,0.463045,0.159881,0.802928,0.438278,0.463192,0.548091,0.388213,0.378023,0.495891,0.288580,0.488314,0.653325,0.411552,0.365625,0.359362,...,0.607268,nonhiphop,1.0,1.0,folk,False,nonhiphop,True,pop,True,nonhiphop,True,folk,False,nonhiphop,True,3.0,False,nonhiphop,True,0 0 pop,NaN,NaN,NaN,NaN
2,mSOdaq656ff,rock,2015-11-11,2015.833333,2015.0,601.0,0.553648,0.305978,0.255246,0.479821,0.377546,0.099607,0.824299,0.524473,0.405405,0.709515,0.526825,0.493619,0.486819,0.663650,0.652753,0.769993,0.717449,0.484404,0.558664,...,0.510593,nonhiphop,0.0,0.0,rock,True,nonhiphop,True,rock,True,nonhiphop,True,rock,True,nonhiphop,True,1.0,False,nonhiphop,True,0 0 rock,NaN,NaN,NaN,NaN
3,bqv5oMs3aac9,pop,2016-05-24,2016.333333,2016.0,425.0,0.317953,0.153784,0.248175,0.558457,0.349581,0.093165,0.757307,0.546043,0.536335,0.520767,0.543061,0.461001,0.461963,0.549711,0.498131,0.638340,0.569084,0.339578,0.534756,...,0.486048,nonhiphop,0.0,0.0,folk,False,nonhiphop,True,pop,True,nonhiphop,True,pop,True,nonhiphop,True,1.0,False,nonhiphop,True,0 0 pop,NaN,NaN,NaN,NaN
4,xLBr1fbf8ac,pop,2015-06-27,2015.416667,2015.0,42450.0,0.295309,0.174921,0.247695,0.550785,0.362049,0.090020,0.749085,0.521823,0.549510,0.566503,0.522691,0.396086,0.530403,0.533256,0.584893,0.616224,0.567201,0.407748,0.466743,...,0.591887,nonhiphop,0.0,0.0,folk,False,nonhiphop,True,pop,True,nonhiphop,True,pop,True,nonhiphop,True,1.0,False,nonhiphop,True,0 0 pop,NaN,NaN,NaN,NaN
5,bqwPhDt3af1d,pop,2016-04-14,2016.250000,2016.0,352.0,0.359844,0.161044,0.276050,0.628411,0.440350,0.079107,0.746165,0.476319,0.578945,0.551392,0.510643,0.440828,0.633674,0.502060,0.614375,0.721540,0.639329,0.453601,0.532991,...,0.470208,nonhiphop,0.0,0.0,pop,True,nonhiphop,True,pop,True,nonhiphop,True,pop,True,nonhiphop,True,1.0,False,nonhiphop,True,0 0 pop,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xOoghub575e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
xOoghvd2ec5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
xOoghwbc25a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
xOoghya0b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
